# 1. Import Necessary Libraries

In [ ]:
# Install necessary packages
!pip install snowflake-connector-python
!pip install snowflake-sqlalchemy
!pip install sqlalchemy
!pip install pandas
!pip install xgboost
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/snowflake/sqlalchemy/base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


# 2. Connect to Snowflake and Load Data

In [ ]:
ACCOUNT = <account>
USER = <user>
PASSWORD = <password>
WAREHOUSE = <warehouse>
DATABASE = <database>
SCHEMA = <schema>

# Create connection URL
engine = create_engine(URL(
    account=ACCOUNT,
    user=USER,
    password=PASSWORD,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA,
))

# Create a connection
connection = engine.connect()

# Load data into pandas DataFrames
customers = pd.read_sql_query("SELECT * FROM customer_demographic", connection)
prospective_buyers = pd.read_sql_query("SELECT * FROM prospect_demographic", connection)

# Close the connection
connection.close()

In [ ]:
customers.head()

,age,maritalstatus,gender,yearlyincome,totalchildren,numberchildrenathome,education,occupation,houseownerflag,numbercarsowned,country,segment
0,43,M,M,90000,2,0,Bachelors,Professional,1,0,Australia,Best Customers
1,38,S,M,60000,3,3,Bachelors,Professional,0,1,Australia,Best Customers
2,43,M,M,60000,3,3,Bachelors,Professional,1,1,Australia,Best Customers
3,41,S,F,70000,0,0,Bachelors,Professional,0,1,Australia,Best Customers
4,35,S,F,80000,5,5,Bachelors,Professional,1,4,Australia,Best Customers


# 3. Data Preprocessing

In [ ]:
# Select Relevant Features
features = [
    'age', 'maritalstatus', 'gender', 'yearlyincome','totalchildren',
    'numbercarsowned', 'education', 'occupation', 'houseownerflag',
    'numberchildrenathome', 'country', 'segment'
]

# Check for missing values
print(customers.isnull().sum())
# Drop rows with missing values
customers.dropna(inplace=True)

age                     0
maritalstatus           0
gender                  0
yearlyincome            0
totalchildren           0
numberchildrenathome    0
education               0
occupation              0
houseownerflag          0
numbercarsowned         0
country                 0
segment                 0
dtype: int64


In [ ]:
# Encode Categorical Variables
categorical_vars = ['maritalstatus', 'gender',
                    'education', 'occupation', 'country']

# Perform one-hot encoding
customers_encoded = pd.get_dummies(customers, columns=categorical_vars, drop_first=True)

# 4. Prepare Data for Modeling

In [ ]:
# Add target
customers_encoded['segment'] = customers['segment']

# Separate features and target variable
X = customers_encoded.drop('segment', axis=1)
y = customers_encoded['segment']

# Encode Target Variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

#  5. Stratified Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 6. Train XGBoost Model

In [ ]:
# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_encoded)), random_state=42)

# Train the model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# 7. Evaluate the Model

In [ ]:
# Evaluate model performance
print("Classification Report with All Features:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Classification Report with All Features:
                 precision    recall  f1-score   support

        At Risk       0.43      0.37      0.39       924
 Best Customers       0.38      0.24      0.29       370
Loyal Customers       0.38      0.25      0.30       554
          Other       0.65      0.81      0.72      1849

       accuracy                           0.56      3697
      macro avg       0.46      0.42      0.43      3697
   weighted avg       0.53      0.56      0.53      3697



# 8. Display Feature Importance

In [ ]:
import plotly.express as px
import pandas as pd

# Get the feature importance from the model
importance_values = xgb_model.get_booster().get_score(importance_type="weight")
importance_df = pd.DataFrame({
    'Feature': list(importance_values.keys()),
    'Importance': list(importance_values.values())
})

# Sort by importance and select top 10
importance_df = importance_df.sort_values(by='Importance', ascending=False).head(10)

# Create a bar chart using plotly
fig = px.bar(
    importance_df,
    x='Importance',
    y='Feature',
    orientation='h',  # Horizontal bar chart
    title='Feature Importance - Top 10'
)

# Customize layout
fig.update_layout(
    title_x=0.5,  # Center the title
    width=800, height=600,  # Set figure size
    yaxis={'categoryorder': 'total ascending'},  # Ensure the bars are sorted by importance
)

# Show the figure
fig.show()

# 9. Evaluate Performance with Only Top Features

In [ ]:
# Get feature importance as a dictionary
feature_importance = xgb_model.get_booster().get_score(importance_type='weight')

# Sort features by importance
sorted_features = sorted(feature_importance, key=feature_importance.get, reverse=True)

# Select top features (let's say top 5 for this experiment)
top_features = sorted_features[:5]
print(f"Top 5 Features: {top_features}")

# Train and evaluate the model with only top features
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

# Initialize the XGBoost classifier with top features
xgb_model_top = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_encoded)), random_state=42)

# Train the model on the reduced set of features
xgb_model_top.fit(X_train_top, y_train)

# Make predictions
y_pred_top = xgb_model_top.predict(X_test_top)

# Evaluate model performance with top features
print("Classification Report with Top Features:")
print(classification_report(y_test, y_pred_top, target_names=le.classes_))

Top 5 Features: ['age', 'yearlyincome', 'numbercarsowned', 'totalchildren', 'numberchildrenathome']
Classification Report with Top Features:
                 precision    recall  f1-score   support

        At Risk       0.38      0.28      0.32       924
 Best Customers       0.32      0.16      0.21       370
Loyal Customers       0.34      0.13      0.19       554
          Other       0.60      0.84      0.70      1849

       accuracy                           0.53      3697
      macro avg       0.41      0.35      0.36      3697
   weighted avg       0.48      0.53      0.48      3697



# 10. Evaluate Performance by Removing Low Importance Features

In [ ]:
# Select bottom features (let's say bottom 5 for this experiment)
bottom_features = sorted_features[-5:]
print(f"Bottom 5 Features: {bottom_features}")

# Remove bottom features from the training and testing sets
X_train_removed = X_train.drop(columns=bottom_features)
X_test_removed = X_test.drop(columns=bottom_features)

# Train and evaluate the model without bottom features
xgb_model_removed = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_encoded)), random_state=42)

# Train the model on the data without low-importance features
xgb_model_removed.fit(X_train_removed, y_train)

# Make predictions
y_pred_removed = xgb_model_removed.predict(X_test_removed)

# Evaluate model performance without bottom features
print("Classification Report without Low Importance Features:")
print(classification_report(y_test, y_pred_removed, target_names=le.classes_))

Bottom 5 Features: ['occupation_Professional', 'education_Partial High School', 'country_France', 'occupation_Management', 'occupation_Manual']
Classification Report without Low Importance Features:
                 precision    recall  f1-score   support

        At Risk       0.42      0.34      0.38       924
 Best Customers       0.33      0.21      0.25       370
Loyal Customers       0.37      0.24      0.29       554
          Other       0.65      0.82      0.72      1849

       accuracy                           0.55      3697
      macro avg       0.44      0.40      0.41      3697
   weighted avg       0.52      0.55      0.53      3697



In [ ]:
import plotly.graph_objects as go

# Data
models = ['Full Model', 'Top Features Only', 'Without Low Features']
accuracies = [accuracy_score(y_test, y_pred),
              accuracy_score(y_test, y_pred_top),
              accuracy_score(y_test, y_pred_removed)]

# Create a bar chart using plotly
fig = go.Figure(data=[go.Bar(x=models, y=accuracies)])

# Customize the layout
fig.update_layout(
    title='Comparison of Model Performance with Feature Selection',
    xaxis_title='Model',
    yaxis_title='Accuracy',
    title_x=0.5,  # Center the title
    width=800, height=600  # Set figure size
)

# Show the figure
fig.show()